In [70]:
import csv
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [31]:
df = pd.read_csv('pl_results.csv', index_col = 0)

<ipython-input-31-ba411fa52566>:1: DtypeWarning: Columns (2,15,16,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pl_results.csv', index_col = 0)


In [6]:
df.shape

(26328, 33)

In [ ]:
df.head()

In [9]:
columns_list = df.columns.tolist()

print("Column names:", columns_list)

Column names: ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes', 'season', 'team', 'date_downloaded', 'Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GD', 'Pts', 'Pts/MP', 'Top Team Scorer', 'Goalkeeper']


In [32]:
df = df[df['Comp'] == 'Premier League'].copy()

In [13]:
df.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Squad,MP,W,D,L,GD,Pts,Pts/MP,Top Team Scorer,Goalkeeper
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
drop_cols = ["Time","Comp" ,"Round", 'Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GD', 'Pts', 'Pts/MP', 'Top Team Scorer', 'Goalkeeper', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes', 'date_downloaded']
df = df.drop(drop_cols, axis=1)

In [35]:
df.head(15)

,Date,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,season,team
1,2022-08-07,Sun,Away,W,2,0,West Ham,2.2,0.5,75.0,2022,Manchester City
2,2022-08-13,Sat,Home,W,4,0,Bournemouth,1.7,0.1,67.0,2022,Manchester City
3,2022-08-21,Sun,Away,D,3,3,Newcastle Utd,2.1,1.8,69.0,2022,Manchester City
4,2022-08-27,Sat,Home,W,4,2,Crystal Palace,2.2,0.1,74.0,2022,Manchester City
5,2022-08-31,Wed,Home,W,6,0,Nott'ham Forest,3.3,0.7,74.0,2022,Manchester City
6,2022-09-03,Sat,Away,D,1,1,Aston Villa,2.1,0.3,71.0,2022,Manchester City
9,2022-09-17,Sat,Away,W,3,0,Wolves,1.1,0.4,59.0,2022,Manchester City
10,2022-10-02,Sun,Home,W,6,3,Manchester Utd,3.2,1.7,53.0,2022,Manchester City
12,2022-10-08,Sat,Home,W,4,0,Southampton,2.4,0.2,65.0,2022,Manchester City
14,2022-10-16,Sun,Away,L,0,1,Liverpool,1.0,2.1,63.0,2022,Manchester City


In [36]:
df['Month'] = pd.to_datetime(df['Date']).dt.month


In [40]:
points = {'W': 3, 'D': 1, 'L': 0}
df['Points'] = df['Result'].replace(points)

In [42]:
df.head(20)

,Date,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,season,team,Month,Points
1,2022-08-07,Sun,Away,W,2,0,West Ham,2.2,0.5,75.0,2022,Manchester City,8,3
2,2022-08-13,Sat,Home,W,4,0,Bournemouth,1.7,0.1,67.0,2022,Manchester City,8,3
3,2022-08-21,Sun,Away,D,3,3,Newcastle Utd,2.1,1.8,69.0,2022,Manchester City,8,1
4,2022-08-27,Sat,Home,W,4,2,Crystal Palace,2.2,0.1,74.0,2022,Manchester City,8,3
5,2022-08-31,Wed,Home,W,6,0,Nott'ham Forest,3.3,0.7,74.0,2022,Manchester City,8,3
6,2022-09-03,Sat,Away,D,1,1,Aston Villa,2.1,0.3,71.0,2022,Manchester City,9,1
9,2022-09-17,Sat,Away,W,3,0,Wolves,1.1,0.4,59.0,2022,Manchester City,9,3
10,2022-10-02,Sun,Home,W,6,3,Manchester Utd,3.2,1.7,53.0,2022,Manchester City,10,3
12,2022-10-08,Sat,Home,W,4,0,Southampton,2.4,0.2,65.0,2022,Manchester City,10,3
14,2022-10-16,Sun,Away,L,0,1,Liverpool,1.0,2.1,63.0,2022,Manchester City,10,0


In [44]:
df.shape

(23330, 14)

In [ ]:
df["team"].value_counts()

In [52]:
df["Venue_Code"] = df["Venue"].astype("category").cat.codes
df["Opponent_Code"] = df["Opponent"].astype("category").cat.codes
df["Day_Code"] = df["Day"].astype("category").cat.codes

In [50]:
del df["Opponenet_Code"]

In [53]:
df.head()

,Date,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,season,team,Month,Points,Venue_Code,Opponent_Code,Day_Code
1,2022-08-07,Sun,Away,W,2,0,West Ham,2.2,0.5,75.0,2022,Manchester City,8,3,0,46,3
2,2022-08-13,Sat,Home,W,4,0,Bournemouth,1.7,0.1,67.0,2022,Manchester City,8,3,1,7,2
3,2022-08-21,Sun,Away,D,3,3,Newcastle Utd,2.1,1.8,69.0,2022,Manchester City,8,1,0,29,3
4,2022-08-27,Sat,Home,W,4,2,Crystal Palace,2.2,0.1,74.0,2022,Manchester City,8,3,1,16,2
5,2022-08-31,Wed,Home,W,6,0,Nott'ham Forest,3.3,0.7,74.0,2022,Manchester City,8,3,1,31,6


ML Stuff Starts here

In [55]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [56]:
training_set = df[df["Date"] < '2021-01-01']
test_set = df[df["Date"] > '2021-01-01']

In [57]:
predictors = ["Month", "Venue_Code", "Opponent_Code", "Day_Code", "xG", "xGA", "Poss"]

In [59]:
x_train = training_set[predictors]
y_train = training_set["Points"]

In [63]:
imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train)

In [64]:
rf.fit(x_train_imputed, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [68]:
preds = rf.predict(test_set[predictors])
error = accuracy_score(test_set["Points"], preds)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [69]:
error

0.6053054662379421

In [73]:
combined = pd.DataFrame(dict(actual=test_set["Points"], predicted=preds))

In [74]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1,3
actual,,,
0,353,31,92
1,126,22,121
3,90,31,378
